In [1]:
import pyspark
import numpy as np
import pandas as pd
import time
import sys

sc = pyspark.SparkContext('local[*]')

In [2]:
sc.defaultParallelism

2

In [3]:
class DataUtil:
    def __init__(self, sc, inputPath='data/spam.data.txt', statsInputPath='data/mean_std.txt', standardize=True):
        self.inputPath = inputPath
        self.standardize = standardize
        self.mean_std = pd.read_csv(statsInputPath, delimiter=',', header=None)
        self.mean = sc.broadcast(self.mean_std.iloc[:,0].values.astype(float))
        self.std = sc.broadcast(self.mean_std.iloc[:,1].values.astype(float))
        
        
    def read(self, sc):
        if self.standardize:
            return sc.textFile(self.inputPath).map(lambda x: np.asarray(x.split(' ')).astype(float)) \
                    .map(lambda x: (x[:56], x[57])) \
                    .map(lambda x: ((x[0] - self.mean.value[:56])/self.std.value[:56], x[1]))
        else:
            return sc.textFile(self.inputPath).map(lambda x: np.asarray(x.split(' ')).astype(float)) \
                    .map(lambda x: (x[:56], x[57]))

In [4]:
spam_dataset = DataUtil(sc, 'data/spam.data.txt', 'data/mean_std.txt', True).read(sc)
train_set, test_set = spam_dataset.randomSplit(weights=[0.8, 0.2], seed=1)

print("Whole dataset: {}, {}".format(spam_dataset.count(), len(spam_dataset.first()[0])))
print("Train dataset: {}, {}".format(train_set.count(), len(train_set.first()[0])))
print("Test dataset: {}, {}".format(test_set.count(), len(test_set.first()[0])))

Whole dataset: 4601, 56
Train dataset: 3675, 56
Test dataset: 926, 56


In [5]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def cost_function(y, h):
    eps = sys.float_info.epsilon
    return y * np.log(h + eps) + (1 - y) * np.log(1 - h + eps)

def stats(data):
    def f(y, pred):
        if y == pred:
            return 1,0,0,0 if y == 1 else 0,1,0,0
        elif pred == 1:
            return 0,0,1,0
        else:
            return 0,0,0,1
    tp, tn, fp, fn = data.map(lambda x: f(x[2], x[4])).reduce(lambda a, b: tuple(map(sum, zip(a, b))))
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    return precision, recall, f1, accuracy

In [6]:
class ParallelLogReg():
    
    def __init__(self, data, iterations, learning_rate, lambda_reg):
        self.data = data
        self.numberObservations = self.data.count()
        self.numberFeatures = len(self.data.first()[0])
        self.iterations = iterations
        self.lr = learning_rate
        self.lambda_reg = lambda_reg
        
    def __add_intercept(self, rdd):
        return rdd.map(lambda x: (1, x[0], x[1]))
        
    def train(self, train_rdd=None, SGD=False, SGD_pct=0.5, threshold=0.5):
        eps = sys.float_info.epsilon
        
        # initialize the weights
        # w[0]: bias weight
        # w[1]: rest of the weights
        w = (0, np.zeros(self.numberFeatures))
        
        if train_rdd == None:
            data = self.__add_intercept(self.data)
        else:
            data = self.__add_intercept(train_rdd)

        # initialize prediction to rdd
        # x[0]: bias/intercept
        # x[1]: rest features
        # x[2]: true y
        # adding x[3]: predicted y
        data = data.map(lambda x: (x[0], x[1], x[2], sigmoid(w[1].dot(x[1]) + w[0] * x[0])))
        numObs = data.count()
        
        train = data.cache()
        
        for i in range(self.iterations):
            start = time.time()
            
            # sample for SGD
            if SGD:
                train = data.sample(False, SGD_pct).repartition(sc.defaultParallelism).cache()
                if i == 0:
                    numObs = train.count()
            
            # compute derivatives
            temp = train.map(lambda x: ((x[3] - x[2]) * x[0], (x[3] - x[2]) * x[1])) \
                       .reduce(lambda a,b: (a[0] + b[0], a[1] + b[1]))
            dw = (temp[0]/numObs, (temp[1]/numObs) + self.lambda_reg * np.abs(w[1]))
            
            # update weights
            w = (w[0] - self.lr * dw[0], w[1] - self.lr * dw[1])
            
            # update prediction
            train = train.map(lambda x: (x[0], x[1], x[2], sigmoid(w[1].dot(x[1]) + w[0] * x[0]))).cache()
            
            if (i % 5 == 0):
                end = time.time()
                
                model_stats = self.validate(train, w, add_intercept=False)

                print("Iteration: " + str(i) + ", Total Time: " + str(end - start))
                print('Precision: {}, Recall: {}, F1: {}, Accuracy: {}'.format(model_stats['precision'], model_stats['recall'], model_stats['f1'], model_stats['accuracy']))
                print('Current loss: {}\n'.format(model_stats['loss']))
                
        model_stats = self.validate(data, w, add_intercept=False)
        
        print('Training stats:')
        print('Precision: {}, Recall: {}, F1: {}, Accuracy: {}'.format(model_stats['precision'], model_stats['recall'], model_stats['f1'], model_stats['accuracy']))
        print('Final training loss: {}\n'.format(model_stats['loss']))
            
        return {**model_stats, 'w': w}
    
    def validate(self, val_rdd, w, threshold=0.5, add_intercept=True):
        if add_intercept:
            val_rdd = self.__add_intercept(val_rdd)
        
        numObs = val_rdd.count()
        
        # update prediction
        val_rdd = val_rdd.map(lambda x: (x[0], x[1], x[2], sigmoid(w[1].dot(x[1]) + w[0] * x[0])))
        
        loss = val_rdd.map(lambda x: cost_function(x[2], x[3])) \
                    .reduce(lambda a,b: a + b)
        loss = -(1/numObs) * loss + (self.lambda_reg/2) * np.sum(w[1]**2)
        
        # calculate stats
        val_rdd = val_rdd.map(lambda x: (x[0], x[1], x[2], x[3], 1 if x[3] >= threshold else 0)).cache()
        precision, recall, f1, accuracy = stats(val_rdd)
        return {'loss': loss, 'precision': precision, 'recall': recall, 'f1': f1, 'accuracy': accuracy}
    
    def cross_validate(self, k, SGD=False, SGD_pct = 0.5):
        
        if k == 1:
            print("Please choose a k > 1. Or use regular train function")
            return
        
        dataWithIndex = self.data.zipWithIndex();
        
        model_stats = []
        models = []
        
        for i in range(k):
            print("Fold: " + str(i))
            
            fold = dataWithIndex.filter(lambda x: x[1] % k != i).repartition(sc.defaultParallelism) \
                                .map(lambda x: x[0])
            
            model = self.train(fold, SGD, SGD_pct)
            models.append(model)
            
            val = dataWithIndex.filter(lambda x: x[1] % k == i).repartition(sc.defaultParallelism) \
                                .map(lambda x: x[0])
            
            val_stats = self.validate(val, model['w'])
            model_stats.append(val_stats)
            print('Validation stats:')
            print('Precision: {}, Recall: {}, F1: {}, Accuracy: {}'.format(val_stats['precision'], val_stats['recall'], val_stats['f1'], val_stats['accuracy']))
            print('Loss: {}\n'.format(val_stats['loss']))

        avg_model_stats = {}
        for key in model_stats[0].keys():
            avg_model_stats['avg_' + key] = sum(stat[key] for stat in model_stats) / len(model_stats)

        print('Averaged validation stats:')
        print('Precision: {}, Recall: {}, F1: {}, Accuracy: {}'.format(avg_model_stats['avg_precision'], avg_model_stats['avg_recall'], avg_model_stats['avg_f1'], avg_model_stats['avg_accuracy']))
        print('Loss: {}'.format(avg_model_stats['avg_loss']))

In [7]:
logReg = ParallelLogReg(data=train_set, iterations=50, learning_rate=0.2, lambda_reg=0.1)

In [8]:
start = time.time()
logReg.cross_validate(10)
print('Total time:')
print(time.time() - start)

Fold: 0
Training stats:
Precision: 0.9541745134965474, Recall: 0.9617209743751978, F1: 0.9579328816763826, Accuracy: 0.9192621711521016
Final training loss: 0.4155793833480021

Validation stats:
Precision: 0.9385474860335196, Recall: 0.9710982658959537, F1: 0.9545454545454546, Accuracy: 0.9130434782608695
Loss: 0.4462898982098461

Fold: 1
Training stats:
Precision: 0.9541745134965474, Recall: 0.9617209743751978, F1: 0.9579328816763826, Accuracy: 0.9192621711521016
Final training loss: 0.42126326573826345

Validation stats:
Precision: 0.9473684210526315, Recall: 0.9799426934097422, F1: 0.9633802816901408, Accuracy: 0.9293478260869565
Loss: 0.4107424088659815

Fold: 2
Training stats:
Precision: 0.9520676691729323, Recall: 0.9635383639822448, F1: 0.9577686731799558, Accuracy: 0.9189597822800121
Final training loss: 0.42215690599657574

Validation stats:
Precision: 0.957983193277311, Recall: 0.9688385269121813, F1: 0.9633802816901408, Accuracy: 0.9293478260869565
Loss: 0.41561123258516497


In [8]:
model = logReg.train()

Training stats:
Precision: 0.9540456723992106, Recall: 0.9635535307517085, F1: 0.958776030599235, Accuracy: 0.9208163265306123
Final training loss: 0.4199384531598891



In [19]:
for i in range(21):
    model_stats = logReg.validate(test_set, model['w'], threshold=i/20)
    print('{}'.format(model_stats['recall']))


1.0
0.998003992015968
0.9944649446494465
0.993322203672788
0.9937106918238994
0.9897810218978103
0.989100817438692
0.9842931937172775
0.9792682926829268
0.9660421545667447
0.9586206896551724
0.9378531073446328
0.9178852643419573
0.9049217002237137
0.8694690265486725
0.8371837183718371
0.812910284463895
0.7726775956284153
0.7442872687704026
0.6916395222584147
0.6198704103671706
